# SAS Viya Workbench를 활용한 Python 모형 적합 절차

## 1. 사전 작업

### 1.1. Packages import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier as python_rf, GradientBoostingClassifier as python_gbm

### 1.2. CSV 데이터 가져오기

In [ ]:
hmeq = pd.read_csv('/workspaces/myfolder/svw-demo/data/hmeq_learn.csv')
hmeq.columns = hmeq.columns.str.upper()
hmeq.head()

## 2. 전처리 및 데이터 분할

### 2.1. One-hot encoding

In [ ]:
# a) 변수 변환
hmeq_dummy = pd.get_dummies(data=hmeq, columns=['REASON', 'JOB'], drop_first=False)

# b) 변환 전 변수 제거 (존재하지 않을 경우에도 오류 없이 처리)
hmeq_dummy = hmeq_dummy.drop(columns=['REASON', 'JOB'], errors='ignore')

# c) 결과 확인
hmeq_dummy.info()

### 2.2. 데이터 분할

#### 1) 데이터 분할 및 편향 변수 제외

In [ ]:
hmeq_train = hmeq_dummy[hmeq_dummy['_PARTIND_'] == 0].drop(columns=['RACE', 'SEX'], errors='ignore')
print("## train  size:", hmeq_train.shape[0] )

#### 2) 입력과 반응 변수 분할

In [ ]:
hmeq_train_x = hmeq_train.drop(columns=['BAD','ID','_PARTIND_'], errors='ignore')
hmeq_train_y = hmeq_train['BAD']

## 3. 모형 적합 및 스코어링

### 3.1. Python 모형 학습

In [ ]:
# a) Random forest Model
pyrf=python_rf()
pyrf.fit(hmeq_train_x, hmeq_train_y)

# b) Gradient Boosting
pygb = python_gbm()
pygb.fit(hmeq_train_x, hmeq_train_y)

### 3.2. 학습 모형 스코어링

#### 1) 스코어링

In [ ]:
pyrf_scored = pd.DataFrame(pyrf.predict_proba(hmeq_dummy.drop(columns=['BAD','ID','_PARTIND_','SEX','RACE'], errors='ignore')), columns=['P_BAD0', 'P_BAD1'])
pygb_scored = pd.DataFrame(pygb.predict_proba(hmeq_dummy.drop(columns=['BAD','ID','_PARTIND_','SEX','RACE'], errors='ignore')), columns=['P_BAD0', 'P_BAD1'])

#### 2) 편향 및 파티션등 변수 추가

In [ ]:
pyrf_pred = pd.concat([hmeq_dummy[['ID', 'BAD', 'SEX', 'RACE', '_PARTIND_']], pyrf_scored], axis=1)
pygb_pred = pd.concat([hmeq_dummy[['ID', 'BAD', 'SEX', 'RACE', '_PARTIND_']], pygb_scored], axis=1)

## 4. 스코어링 데이터 내보내기

In [ ]:
pyrf_pred.to_csv('/workspaces/svwdemo/result/hmeq_pyrf_pred.csv', index=False)
pygb_pred.to_csv('/workspaces/svwdemo/result/hmeq_pygb_pred.csv', index=False)